# Bitcoin Data Checker

Simple notebook to check Bitcoin option data, specifically looking at **BTC-28JUN24-58000-C** around 12:30 to see when the bid goes null.

## Objectives
- Load orderbook data quickly
- Filter for specific option symbol  
- Check for null bid prices around 12:30
- Show basic price information

In [1]:
# Import libraries
import polars as pl
from datetime import datetime

print("📦 Libraries loaded")

📦 Libraries loaded


In [2]:
# Configuration
target_symbol = "BTC-28JUN24-58000-C"
data_file = "../data_orderbook/20240229.output.csv.gz"

print(f"🎯 Target: {target_symbol}")
print(f"📁 Data: {data_file}")

🎯 Target: BTC-28JUN24-58000-C
📁 Data: ../data_orderbook/20240229.output.csv.gz


In [3]:
# Load data
try:
    print("📊 Loading data...")
    df = pl.scan_csv(data_file)
    
    # Test read
    sample = df.head(1).collect()
    print(f"✅ Data loaded, columns: {sample.columns}")
    
except Exception as e:
    print(f"❌ Error: {e}")

📊 Loading data...
✅ Data loaded, columns: ['symbol', 'timestamp', 'exchange', 'local_timestamp', 'asks[0].price', 'asks[0].amount', 'bids[0].price', 'bids[0].amount', 'asks[1].price', 'asks[1].amount', 'bids[1].price', 'bids[1].amount', 'asks[2].price', 'asks[2].amount', 'bids[2].price', 'bids[2].amount', 'asks[3].price', 'asks[3].amount', 'bids[3].price', 'bids[3].amount', 'asks[4].price', 'asks[4].amount', 'bids[4].price', 'bids[4].amount', 'index_price']


In [4]:
sample

symbol,timestamp,exchange,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,bids[1].amount,asks[2].price,asks[2].amount,bids[2].price,bids[2].amount,asks[3].price,asks[3].amount,bids[3].price,bids[3].amount,asks[4].price,asks[4].amount,bids[4].price,bids[4].amount,index_price
str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC-PERPETUAL""",1709164810000000,"""deribit""",1709164810043745,62488.5,102300.0,62488.0,22500.0,62489.0,78010.0,62486.0,10000.0,62490.0,22410.0,62484.5,20.0,62490.5,43490.0,62479.0,10.0,62492.0,5990.0,62473.5,20.0,62490.36


In [5]:
# Filter for target symbol around 12:30
try:
    print(f"🔍 Filtering for {target_symbol}...")
    
    target_data = (
        df
        .with_columns(pl.col('timestamp').cast(pl.Datetime(time_unit='us')).alias('dt'))
        .filter(
            (pl.col('symbol') == target_symbol) &
            (pl.col('dt').dt.hour() == 12) &
            (pl.col('dt').dt.minute().is_between(32, 35))
        )
        .select(['dt', 'symbol', 'bids[0].price', 'asks[0].price'])
        .collect()
    )
    
    print(f"📊 Found {len(target_data)} records")
    
    if len(target_data) > 0:
        print(target_data)
    else:
        print("⚠️ No data found")
        
except Exception as e:
    print(f"❌ Error filtering: {e}")

🔍 Filtering for BTC-28JUN24-58000-C...
📊 Found 19 records
shape: (19, 4)
┌─────────────────────┬─────────────────────┬───────────────┬───────────────┐
│ dt                  ┆ symbol              ┆ bids[0].price ┆ asks[0].price │
│ ---                 ┆ ---                 ┆ ---           ┆ ---           │
│ datetime[μs]        ┆ str                 ┆ f64           ┆ f64           │
╞═════════════════════╪═════════════════════╪═══════════════╪═══════════════╡
│ 2024-02-29 12:32:00 ┆ BTC-28JUN24-58000-C ┆ 0.2095        ┆ 0.2125        │
│ 2024-02-29 12:32:10 ┆ BTC-28JUN24-58000-C ┆ 0.2095        ┆ 0.2125        │
│ 2024-02-29 12:32:20 ┆ BTC-28JUN24-58000-C ┆ 0.2095        ┆ 0.2125        │
│ 2024-02-29 12:32:30 ┆ BTC-28JUN24-58000-C ┆ 0.2095        ┆ 0.2125        │
│ 2024-02-29 12:32:40 ┆ BTC-28JUN24-58000-C ┆ 0.2095        ┆ null          │
│ …                   ┆ …                   ┆ …             ┆ …             │
│ 2024-02-29 12:34:20 ┆ BTC-28JUN24-58000-C ┆ 0.18          ┆ null   

In [6]:
# Check for null asks
if 'target_data' in locals() and len(target_data) > 0:
    try:
        print("🔍 Checking for null asks...")
        
        # Check null asks
        null_asks = target_data.filter(pl.col('asks[0].price').is_null())
        
        print(f"📊 Records with null ask: {len(null_asks)}")
        
        if len(null_asks) > 0:
            print("🎯 Null ask times:")
            for row in null_asks.iter_rows(named=True):
                print(f"   • {row['dt']}: ask=NULL, ask={row['asks[0].price']}")
        else:
            print("✅ No null asks found in this time window")
            
    except Exception as e:
        print(f"❌ Error checking nulls: {e}")
else:
    print("❌ No target data to analyze")

🔍 Checking for null asks...
📊 Records with null ask: 15
🎯 Null ask times:
   • 2024-02-29 12:32:40: ask=NULL, ask=None
   • 2024-02-29 12:32:50: ask=NULL, ask=None
   • 2024-02-29 12:33:00: ask=NULL, ask=None
   • 2024-02-29 12:33:10: ask=NULL, ask=None
   • 2024-02-29 12:33:20: ask=NULL, ask=None
   • 2024-02-29 12:33:30: ask=NULL, ask=None
   • 2024-02-29 12:33:40: ask=NULL, ask=None
   • 2024-02-29 12:33:50: ask=NULL, ask=None
   • 2024-02-29 12:34:00: ask=NULL, ask=None
   • 2024-02-29 12:34:10: ask=NULL, ask=None
   • 2024-02-29 12:34:20: ask=NULL, ask=None
   • 2024-02-29 12:34:30: ask=NULL, ask=None
   • 2024-02-29 12:34:40: ask=NULL, ask=None
   • 2024-02-29 12:34:50: ask=NULL, ask=None
   • 2024-02-29 12:35:00: ask=NULL, ask=None


In [7]:
# Set polars to show 30 rows
pl.Config.set_tbl_rows(30)

polars.config.Config

In [8]:
# DeribitMDManager get_conflated_md() Analysis
import sys
sys.path.append('../')
from btc_options.data_managers.orderbook_deribit_md_manager import OrderbookDeribitMDManager, DeribitMDManager

print("🔧 Creating OrderbookDeribitMDManager...")
# data_file = "../data_orderbook/20240229.output.csv.gz"
df = pl.scan_csv(data_file)
symbol_manager = OrderbookDeribitMDManager(df, "20240229")
# symbol_manager = DeribitMDManager(df, "20240229")

print("📊 Running get_conflated_md()...")
df_conflated_md = symbol_manager.get_conflated_md(freq='1m', period='5m')

# Filter for our target symbol
conflated_data = df_conflated_md.filter(pl.col('symbol') == target_symbol)
print(f"✅ Found {len(conflated_data)} conflated records for {target_symbol}")

# Get snapshot around 12:30
snapshot_data = conflated_data.filter(
    (pl.col('timestamp').dt.hour() == 12) &
    (pl.col('timestamp').dt.minute().is_between(25, 35))
)

print(f"📊 Conflated records around 12:30: {len(snapshot_data)}")

display(snapshot_data)

🔧 Creating OrderbookDeribitMDManager...
📊 Running get_conflated_md()...
🔄 Converting order book depth data to BBO format (using level 0)...
✅ Converted 8194008 orderbook rows to 8194008 BBO rows
🔄 Normalizing volume from USD to BTC for futures and perpetuals...
✅ Normalized volumes for 8/8 futures/perpetual symbols
✅ Found 1439 conflated records for BTC-28JUN24-58000-C
📊 Conflated records around 12:30: 11


symbol,timestamp,bid_price,ask_price,bid_size,ask_size,S,is_option,is_index,is_perp,is_call,is_put,is_future,expiry,strike,expiry_ts,symbol_fut,bid_price_fut,ask_price_fut,bid_size_fut,ask_size_fut,index_price_fut,tau
str,datetime[μs],f64,f64,f64,f64,f64,bool,bool,bool,bool,bool,bool,str,i64,datetime[μs],str,f64,f64,f64,f64,f64,f64
"""BTC-28JUN24-58000-C""",2024-02-29 12:25:00,0.209,0.212,5.5,9.0,62679.04,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65910.90825,65971.5965,0.8093,0.3742,62646.98,0.328263
"""BTC-28JUN24-58000-C""",2024-02-29 12:26:00,0.209,0.212,5.5,3.0,62721.3,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65945.90475,66019.10125,0.8899,0.6691,62693.01,0.328261
"""BTC-28JUN24-58000-C""",2024-02-29 12:27:00,0.2095,0.2125,4.0,6.0,62733.65,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",66018.3975,66061.6055,0.6805,0.9186,62733.65,0.328259
"""BTC-28JUN24-58000-C""",2024-02-29 12:28:00,0.21,0.2125,5.5,3.0,62760.88,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",66042.5,66084.10775,1.0,0.4584,62760.88,0.328257
"""BTC-28JUN24-58000-C""",2024-02-29 12:29:00,0.209,0.2125,5.6,7.5,62695.25,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65958.4035,66011.6005,0.9251,0.3633,62695.25,0.328255
"""BTC-28JUN24-58000-C""",2024-02-29 12:30:00,0.2095,0.212,0.5,8.9,62688.68,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65958.4035,66006.6,0.8406,0.2278,62691.69,0.328253
"""BTC-28JUN24-58000-C""",2024-02-29 12:31:00,0.2095,0.212,0.5,10.6,62665.29,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65932.5,65969.09625,1.0,0.3685,62665.29,0.328252
"""BTC-28JUN24-58000-C""",2024-02-29 12:32:00,0.2095,0.2125,4.0,7.7,62712.75,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65985.90075,66034.10275,0.6809,0.8918,62712.75,0.32825
"""BTC-28JUN24-58000-C""",2024-02-29 12:33:00,0.2095,null,1.0,null,62725.92,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65655.0,66216.621,1.0,0.4135,62725.92,0.328248


In [9]:
conflated_data.filter(
    (pl.col('timestamp').dt.hour() == 12) &
    (pl.col('timestamp').dt.minute().is_between(25, 39))
)

symbol,timestamp,bid_price,ask_price,bid_size,ask_size,S,is_option,is_index,is_perp,is_call,is_put,is_future,expiry,strike,expiry_ts,symbol_fut,bid_price_fut,ask_price_fut,bid_size_fut,ask_size_fut,index_price_fut,tau
str,datetime[μs],f64,f64,f64,f64,f64,bool,bool,bool,bool,bool,bool,str,i64,datetime[μs],str,f64,f64,f64,f64,f64,f64
"""BTC-28JUN24-58000-C""",2024-02-29 12:25:00,0.209,0.212,5.5,9.0,62679.04,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65910.90825,65971.5965,0.8093,0.3742,62646.98,0.328263
"""BTC-28JUN24-58000-C""",2024-02-29 12:26:00,0.209,0.212,5.5,3.0,62721.3,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65945.90475,66019.10125,0.8899,0.6691,62693.01,0.328261
"""BTC-28JUN24-58000-C""",2024-02-29 12:27:00,0.2095,0.2125,4.0,6.0,62733.65,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",66018.3975,66061.6055,0.6805,0.9186,62733.65,0.328259
"""BTC-28JUN24-58000-C""",2024-02-29 12:28:00,0.21,0.2125,5.5,3.0,62760.88,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",66042.5,66084.10775,1.0,0.4584,62760.88,0.328257
"""BTC-28JUN24-58000-C""",2024-02-29 12:29:00,0.209,0.2125,5.6,7.5,62695.25,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65958.4035,66011.6005,0.9251,0.3633,62695.25,0.328255
"""BTC-28JUN24-58000-C""",2024-02-29 12:30:00,0.2095,0.212,0.5,8.9,62688.68,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65958.4035,66006.6,0.8406,0.2278,62691.69,0.328253
"""BTC-28JUN24-58000-C""",2024-02-29 12:31:00,0.2095,0.212,0.5,10.6,62665.29,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65932.5,65969.09625,1.0,0.3685,62665.29,0.328252
"""BTC-28JUN24-58000-C""",2024-02-29 12:32:00,0.2095,0.2125,4.0,7.7,62712.75,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65985.90075,66034.10275,0.6809,0.8918,62712.75,0.32825
"""BTC-28JUN24-58000-C""",2024-02-29 12:33:00,0.2095,null,1.0,null,62725.92,true,false,false,true,false,false,"""28JUN24""",58000,2024-06-28 08:00:00,"""BTC-28JUN24""",65655.0,66216.621,1.0,0.4135,62725.92,0.328248
